In [1]:
# @title Cell 1: CASME II Key Frame Sequence ViT Infrastructure Configuration

# File: 06_01_ViT_CASME2_KFS_Cell1.py
# Location: experiments/06_01_ViT_CASME2-KFS-PREP.ipynb
# Purpose: ViT-Base for CASME II micro-expression recognition with key frame sequence strategy and face-aware preprocessing

from google.colab import drive
print("=" * 60)
print("CASME II KEY FRAME SEQUENCE ViT WITH FACE-AWARE PREPROCESSING")
print("=" * 60)
print("\n[1] Mounting Google Drive...")
drive.mount('/content/drive')
print("Google Drive mounted successfully")

print("\n[2] Importing required libraries...")
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import torchvision.transforms as transforms
import timm
import json
import os
import numpy as np
import pandas as pd
from PIL import Image
import time
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score
import warnings
warnings.filterwarnings('ignore')

# Project paths configuration
PROJECT_ROOT = "/content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project"
DATASET_ROOT = f"{PROJECT_ROOT}/datasets/processed_casme2/preprocessed_v8"
CHECKPOINT_ROOT = f"{PROJECT_ROOT}/models/06_01_vit_casme2_kfs_prep"
RESULTS_ROOT = f"{PROJECT_ROOT}/results/06_01_vit_casme2_kfs_prep"

# Load CASME II v8 preprocessing metadata
PREPROCESSING_SUMMARY = f"{DATASET_ROOT}/preprocessing_summary.json"

print("CASME II Key Frame Sequence ViT - Face-Aware Preprocessing Infrastructure")
print("=" * 60)

# Validate preprocessing metadata exists
if not os.path.exists(PREPROCESSING_SUMMARY):
    raise FileNotFoundError(f"v8 preprocessing summary not found: {PREPROCESSING_SUMMARY}")

# Load v8 preprocessing metadata
print("Loading CASME II v8 preprocessing metadata...")
with open(PREPROCESSING_SUMMARY, 'r') as f:
    preprocessing_info = json.load(f)

print(f"Dataset variant: {preprocessing_info['variant']}")
print(f"Processing date: {preprocessing_info['processing_date']}")
print(f"Preprocessing method: {preprocessing_info['preprocessing_method']}")
print(f"Total images processed: {preprocessing_info['total_processed']}")
print(f"Face detection rate: {preprocessing_info['face_detection_stats']['detection_rate']:.2%}")

# Extract preprocessing parameters
preproc_params = preprocessing_info['preprocessing_parameters']
print(f"Target size: {preproc_params['target_size']}x{preproc_params['target_size']}px")
print(f"BBox expansion: {preproc_params['bbox_expansion']}px (all directions)")

# Display split information
print(f"\nDataset split information:")
print(f"  Train samples: {preprocessing_info['splits']['train']['total_images']}")
print(f"  Validation samples: {preprocessing_info['splits']['val']['total_images']}")
print(f"  Test samples: {preprocessing_info['splits']['test']['total_images']}")

# =====================================================
# EXPERIMENT CONFIGURATION - Key Frame Sequence with Face-Aware Preprocessing
# =====================================================

# FOCAL LOSS CONFIGURATION - Toggle for experimentation
USE_FOCAL_LOSS = True  # Default: CrossEntropy, Set True to enable Focal Loss
FOCAL_LOSS_GAMMA = 2.0  # Focal loss focusing parameter (if enabled)

# OPTIMIZED CLASS WEIGHTS CONFIGURATION
# CASME II classes: ['others', 'disgust', 'happiness', 'repression', 'surprise', 'sadness', 'fear']
# v8 Train distribution: [237, 150, 75, 63, 60, 15, 3] - inverse sqrt frequency approach

# CrossEntropy Loss - Optimized inverse square root frequency weights
CROSSENTROPY_CLASS_WEIGHTS = [1.00, 1.26, 1.78, 1.94, 1.99, 3.98, 8.90]

# Focal Loss - Normalized per-class alpha values (sum = 1.0)
FOCAL_LOSS_ALPHA_WEIGHTS = [0.055, 0.069, 0.098, 0.107, 0.109, 0.219, 0.343]

# ViT MODEL CONFIGURATION - Support patch16 and patch32
VIT_MODEL_VARIANT = 'patch16'  # Default: patch32, Options: 'patch16' or 'patch32'

# Dynamic ViT model selection based on patch size
if VIT_MODEL_VARIANT == 'patch16':
    VIT_MODEL_NAME = 'google/vit-base-patch16-224-in21k'
    PATCH_SIZE = 16
    print("Using ViT-Base Patch16 for fine-grained micro-expression analysis")
elif VIT_MODEL_VARIANT == 'patch32':
    VIT_MODEL_NAME = 'google/vit-base-patch32-224-in21k'
    PATCH_SIZE = 32
    print("Using ViT-Base Patch32 for efficient micro-expression recognition")
else:
    raise ValueError(f"Unsupported VIT_MODEL_VARIANT: {VIT_MODEL_VARIANT}")

# Display experiment configuration
print("\n" + "=" * 50)
print("EXPERIMENT CONFIGURATION - KEY FRAME SEQUENCE FACE-AWARE")
print("=" * 50)
print(f"Dataset: v8 Key Frame Sequence with Face-Aware Preprocessing")
print(f"Frame strategy: onset, apex, offset (3 frames per video)")
print(f"Training approach: Frame-level independent learning for late fusion")
print(f"Loss Function: {'Focal Loss' if USE_FOCAL_LOSS else 'CrossEntropy Loss'}")
if USE_FOCAL_LOSS:
    print(f"  Gamma: {FOCAL_LOSS_GAMMA}")
    print(f"  Alpha Weights (per-class): {FOCAL_LOSS_ALPHA_WEIGHTS}")
    print(f"  Alpha Sum Validation: {sum(FOCAL_LOSS_ALPHA_WEIGHTS):.3f}")
else:
    print(f"  Class Weights (inverse sqrt freq): {CROSSENTROPY_CLASS_WEIGHTS}")
print(f"ViT Model: {VIT_MODEL_NAME}")
print(f"Patch Size: {PATCH_SIZE}px")
print(f"Input Resolution: 224x224px (face-centered with bbox expansion)")
print(f"Image Format: Grayscale converted to RGB (3-channel)")
print("=" * 50)

# Enhanced GPU configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
gpu_name = torch.cuda.get_device_name(0) if torch.cuda.is_available() else "CPU"
gpu_memory = torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0

print(f"\nDevice: {device}")
print(f"GPU: {gpu_name} ({gpu_memory:.1f} GB)")

# Optimized batch size for moderate dataset (603 train samples)
# Using batch size 8 for better training stability with 603 samples
BATCH_SIZE = 8
NUM_WORKERS = 4

if 'A100' in gpu_name or 'L4' in gpu_name:
    torch.backends.cudnn.benchmark = True
    print(f"GPU optimization enabled for {gpu_name}")

print(f"Moderate dataset configuration: Batch size {BATCH_SIZE} (optimal for 603 samples)")
print(f"Iterations per epoch: {603 // BATCH_SIZE} (~75 iterations per epoch)")

# RAM preloading workers (separate from DataLoader workers)
RAM_PRELOAD_WORKERS = 32
print(f"RAM preload workers: {RAM_PRELOAD_WORKERS} (parallel image loading)")

# CASME II class mapping and analysis
CASME2_CLASSES = ['others', 'disgust', 'happiness', 'repression', 'surprise', 'sadness', 'fear']
CLASS_TO_IDX = {cls: idx for idx, cls in enumerate(CASME2_CLASSES)}

# Extract class distribution from v8 preprocessing metadata
print("\nLoading v8 class distribution...")
train_dist = preprocessing_info['splits']['train']['emotion_distribution']
val_dist = preprocessing_info['splits']['val']['emotion_distribution']
test_dist = preprocessing_info['splits']['test']['emotion_distribution']

# Convert to ordered list matching CASME2_CLASSES
def emotion_dist_to_list(emotion_dict, class_names):
    """Convert emotion distribution dict to ordered list"""
    return [emotion_dict.get(cls, 0) for cls in class_names]

train_dist_list = emotion_dist_to_list(train_dist, CASME2_CLASSES)
val_dist_list = emotion_dist_to_list(val_dist, CASME2_CLASSES)
test_dist_list = emotion_dist_to_list(test_dist, CASME2_CLASSES)

print(f"\nv8 Train distribution: {train_dist_list}")
print(f"v8 Val distribution: {val_dist_list}")
print(f"v8 Test distribution: {test_dist_list}")

# Apply optimized class weights based on loss function selection
if USE_FOCAL_LOSS:
    class_weights = torch.tensor(FOCAL_LOSS_ALPHA_WEIGHTS, dtype=torch.float32).to(device)
    print(f"Applied Focal Loss alpha weights: {class_weights.cpu().numpy()}")
    print(f"Alpha weights sum: {class_weights.sum().item():.3f}")
else:
    class_weights = torch.tensor(CROSSENTROPY_CLASS_WEIGHTS, dtype=torch.float32).to(device)
    print(f"Applied CrossEntropy class weights: {class_weights.cpu().numpy()}")

# CASME II ViT Configuration for Key Frame Sequence with Face-Aware Preprocessing
# Optimized for moderate dataset (603 train samples) with class imbalance
CASME2_VIT_CONFIG = {
    # Architecture configuration - simplified for regularization
    'vit_model': VIT_MODEL_NAME,
    'patch_size': PATCH_SIZE,
    'input_size': 224,
    'num_classes': 7,
    'dropout_rate': 0.3,  # Increased for stronger regularization
    'expected_feature_dim': 768,
    'interpolate_pos_encoding': False,  # No interpolation needed for native 224px

    # Training configuration - optimized for moderate dataset
    'learning_rate': 2e-5,  # Optimized for small batch size
    'weight_decay': 1e-5,
    'gradient_clip': 1.0,
    'num_epochs': 50,
    'batch_size': BATCH_SIZE,
    'num_workers': NUM_WORKERS,
    'device': device,

    # Scheduler configuration
    'scheduler_type': 'plateau',
    'scheduler_mode': 'max',
    'scheduler_factor': 0.5,
    'scheduler_patience': 5,  # Increased for stability
    'scheduler_min_lr': 1e-6,
    'scheduler_monitor': 'val_f1_macro',

    # Loss configuration
    'use_focal_loss': USE_FOCAL_LOSS,
    'focal_loss_gamma': FOCAL_LOSS_GAMMA,
    'focal_loss_alpha_weights': FOCAL_LOSS_ALPHA_WEIGHTS,
    'crossentropy_class_weights': CROSSENTROPY_CLASS_WEIGHTS,
    'class_weights': class_weights,

    # Evaluation configuration
    'use_macro_avg': True,
    'early_stopping': False,
    'save_best_f1': True,
    'save_strategy': 'best_only',

    # v8 specific configuration
    'dataset_version': 'v8',
    'preprocessing_method': 'face_aware_bbox_expansion',
    'frame_strategy': 'key_frame_sequence',
    'frame_types': ['onset', 'apex', 'offset'],
    'image_format': 'grayscale_to_rgb',
    'bbox_expansion': preproc_params['bbox_expansion'],
    'face_detection_rate': preprocessing_info['face_detection_stats']['detection_rate'],
    'training_approach': 'frame_level_independent',
    'inference_strategy': 'late_fusion'
}

print(f"\nViT Configuration Summary:")
print(f"  Model: {CASME2_VIT_CONFIG['vit_model']}")
print(f"  Input size: {CASME2_VIT_CONFIG['input_size']}px")
print(f"  Learning rate: {CASME2_VIT_CONFIG['learning_rate']} (optimized for small batches)")
print(f"  Batch size: {BATCH_SIZE} (optimal for moderate dataset)")
print(f"  Dropout rate: {CASME2_VIT_CONFIG['dropout_rate']} (increased regularization)")
print(f"  Dataset version: {CASME2_VIT_CONFIG['dataset_version']}")
print(f"  Frame strategy: {CASME2_VIT_CONFIG['frame_strategy']}")
print(f"  Frame types: {CASME2_VIT_CONFIG['frame_types']}")
print(f"  Training approach: {CASME2_VIT_CONFIG['training_approach']}")
print(f"  Inference strategy: {CASME2_VIT_CONFIG['inference_strategy']}")
print(f"  Preprocessing: {CASME2_VIT_CONFIG['preprocessing_method']}")
print(f"  Train frames: {preprocessing_info['splits']['train']['total_images']}")
print(f"  Architecture: Simplified 768->256->7 for regularization")

# =====================================================
# FOCAL LOSS IMPLEMENTATION
# =====================================================

class OptimizedFocalLoss(nn.Module):
    """
    Advanced Focal Loss implementation with per-class alpha support
    Paper: Focal Loss for Dense Object Detection (Lin et al., 2017)
    """

    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(OptimizedFocalLoss, self).__init__()

        if alpha is not None:
            if isinstance(alpha, list):
                self.alpha = torch.tensor(alpha, dtype=torch.float32)
            else:
                self.alpha = alpha

            alpha_sum = self.alpha.sum().item()
            if abs(alpha_sum - 1.0) > 0.01:
                print(f"Warning: Alpha weights sum to {alpha_sum:.3f}, expected 1.0")
        else:
            self.alpha = None

        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        pt = torch.exp(-ce_loss)

        if self.alpha is not None:
            if self.alpha.device != targets.device:
                self.alpha = self.alpha.to(targets.device)
            alpha_t = self.alpha.gather(0, targets)
        else:
            alpha_t = 1.0

        focal_loss = alpha_t * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# ViT Architecture for CASME II Key Frame Sequence
class ViTCASME2Baseline(nn.Module):
    """ViT baseline for CASME II micro-expression recognition - Simplified for regularization"""

    def __init__(self, num_classes, dropout_rate=0.3):
        super(ViTCASME2Baseline, self).__init__()

        from transformers import ViTModel

        self.vit = ViTModel.from_pretrained(
            CASME2_VIT_CONFIG['vit_model'],
            add_pooling_layer=False
        )

        # Enable fine-tuning
        for param in self.vit.parameters():
            param.requires_grad = True

        self.vit_feature_dim = self.vit.config.hidden_size

        print(f"ViT feature dimension: {self.vit_feature_dim}")

        # Simplified classification head for regularization
        # Reduced complexity: 768 -> 256 -> 7 (instead of 768 -> 512 -> 128 -> 7)
        self.classifier_layers = nn.Sequential(
            nn.Linear(self.vit_feature_dim, 256),
            nn.LayerNorm(256),
            nn.GELU(),
            nn.Dropout(dropout_rate),
        )

        self.classifier = nn.Linear(256, num_classes)

        print(f"ViT CASME II Simplified: {self.vit_feature_dim} -> 256 -> {num_classes}")
        print(f"Dropout rate: {dropout_rate} (increased for regularization)")

    def forward(self, pixel_values):
        vit_outputs = self.vit(
            pixel_values=pixel_values,
            interpolate_pos_encoding=CASME2_VIT_CONFIG['interpolate_pos_encoding']
        )

        vit_features = vit_outputs.last_hidden_state[:, 0]
        processed_features = self.classifier_layers(vit_features)
        output = self.classifier(processed_features)

        return output

# Optimizer and scheduler factory
def create_optimizer_scheduler_casme2(model, config):
    """Create optimizer and scheduler for CASME II ViT training"""

    optimizer = optim.AdamW(
        model.parameters(),
        lr=config['learning_rate'],
        weight_decay=config['weight_decay'],
        betas=(0.9, 0.999)
    )

    if config['scheduler_type'] == 'plateau':
        scheduler = optim.lr_scheduler.ReduceLROnPlateau(
            optimizer,
            mode=config['scheduler_mode'],
            factor=config['scheduler_factor'],
            patience=config['scheduler_patience'],
            min_lr=config['scheduler_min_lr']
        )
        print(f"Scheduler: ReduceLROnPlateau monitoring {config['scheduler_monitor']}")
    else:
        scheduler = None

    return optimizer, scheduler

# ViT Image Processor setup for 224px input
from transformers import ViTImageProcessor

print("\nSetting up ViT Image Processor for 224px input...")

vit_processor = ViTImageProcessor.from_pretrained(
    CASME2_VIT_CONFIG['vit_model'],
    do_resize=True,
    size={'height': 224, 'width': 224},
    do_normalize=True,
    do_rescale=True,
    do_center_crop=False
)

# Transform functions for ViT
def vit_transform_train(image):
    """Training transform with ViT Image Processor"""
    inputs = vit_processor(image, return_tensors="pt")
    return inputs['pixel_values'].squeeze(0)

def vit_transform_val(image):
    """Validation transform with ViT Image Processor"""
    inputs = vit_processor(image, return_tensors="pt")
    return inputs['pixel_values'].squeeze(0)

print("ViT Image Processor configured for 224px native resolution")

# Custom Dataset class for CASME II v8 preprocessing
class CASME2Dataset(Dataset):
    """Custom dataset class for CASME II v8 preprocessing with flat directory structure"""

    def __init__(self, dataset_root, split, transform=None):
        self.dataset_root = dataset_root
        self.split = split
        self.transform = transform
        self.images = []
        self.labels = []
        self.filenames = []

        split_path = os.path.join(dataset_root, split)

        if not os.path.exists(split_path):
            raise FileNotFoundError(f"Split directory not found: {split_path}")

        # Load all images from flat directory structure
        print(f"Loading {split} dataset from {split_path}...")

        all_files = [f for f in os.listdir(split_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
        print(f"Found {len(all_files)} image files in directory")

        if len(all_files) > 0:
            print(f"Sample filename: {all_files[0]}")

        loaded_count = 0
        skipped_count = 0

        for filename in sorted(all_files):
            # Extract emotion from filename - flexible approach
            # Try multiple patterns to handle different filename formats
            emotion_found = None

            # Remove file extension first
            name_without_ext = filename.rsplit('.', 1)[0]

            # Pattern 1: Check if any emotion class appears in filename
            for emotion_class in CASME2_CLASSES:
                if emotion_class in name_without_ext.lower():
                    emotion_found = emotion_class
                    break

            if emotion_found and emotion_found in CLASS_TO_IDX:
                image_path = os.path.join(split_path, filename)
                self.images.append(image_path)
                self.labels.append(CLASS_TO_IDX[emotion_found])
                self.filenames.append(filename)
                loaded_count += 1
            else:
                skipped_count += 1
                if skipped_count <= 3:
                    print(f"  Skipped (no emotion found): {filename}")

        print(f"Loaded {len(self.images)} samples for {split} split")
        if skipped_count > 0:
            print(f"  Skipped {skipped_count} files (no recognizable emotion)")

        if len(self.images) == 0:
            print(f"ERROR: No samples loaded! Check filename format and emotion labels.")
            print(f"Expected emotions in filenames: {CASME2_CLASSES}")

        self._print_distribution()

    def _print_distribution(self):
        """Print class distribution"""
        if len(self.labels) == 0:
            print("  No samples to display distribution")
            return

        label_counts = {}
        for label in self.labels:
            label_counts[label] = label_counts.get(label, 0) + 1

        for label, count in sorted(label_counts.items()):
            class_name = CASME2_CLASSES[label]
            percentage = (count / len(self.labels)) * 100
            print(f"  {class_name}: {count} samples ({percentage:.1f}%)")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image_path = self.images[idx]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, self.labels[idx], self.filenames[idx]

# Create directories
os.makedirs(CHECKPOINT_ROOT, exist_ok=True)
os.makedirs(f"{RESULTS_ROOT}/training_logs", exist_ok=True)
os.makedirs(f"{RESULTS_ROOT}/evaluation_results", exist_ok=True)

print(f"\nDataset paths:")
print(f"Train: {DATASET_ROOT}/train")
print(f"Validation: {DATASET_ROOT}/val")
print(f"Test: {DATASET_ROOT}/test")

# Architecture validation
print("\nViT CASME II architecture validation...")

try:
    test_model = ViTCASME2Baseline(num_classes=7, dropout_rate=0.3).to(device)
    test_input = torch.randn(1, 3, 224, 224).to(device)
    test_output = test_model(test_input)

    expected_tokens = (224 // CASME2_VIT_CONFIG['patch_size']) ** 2

    print(f"Validation successful: Output shape {test_output.shape}")
    print(f"Expected tokens for 224px with patch{CASME2_VIT_CONFIG['patch_size']}: {expected_tokens} tokens")

    del test_model, test_input, test_output
    torch.cuda.empty_cache()

except Exception as e:
    print(f"Validation failed: {e}")

# Loss function factory
def create_criterion_casme2(weights, use_focal_loss=False, alpha_weights=None, gamma=2.0):
    """
    Factory function to create loss criterion based on configuration

    Args:
        weights: Class weights for CrossEntropy
        use_focal_loss: Whether to use Focal Loss or CrossEntropy
        alpha_weights: Per-class alpha weights for Focal Loss
        gamma: Focal loss gamma parameter

    Returns:
        Loss function
    """
    if use_focal_loss:
        print(f"Using Optimized Focal Loss with gamma={gamma}")
        if alpha_weights:
            print(f"Per-class alpha weights: {alpha_weights}")
            print(f"Alpha sum: {sum(alpha_weights):.3f}")
        return OptimizedFocalLoss(alpha=alpha_weights, gamma=gamma)
    else:
        print(f"Using CrossEntropy Loss with optimized class weights")
        print(f"Class weights: {weights.cpu().numpy()}")
        return nn.CrossEntropyLoss(weight=weights)

# Global configuration for training pipeline
GLOBAL_CONFIG_CASME2 = {
    'device': device,
    'batch_size': BATCH_SIZE,
    'num_workers': NUM_WORKERS,
    'num_classes': 7,
    'class_weights': class_weights,
    'class_names': CASME2_CLASSES,
    'class_to_idx': CLASS_TO_IDX,
    'transform_train': vit_transform_train,
    'transform_val': vit_transform_val,
    'vit_config': CASME2_VIT_CONFIG,
    'checkpoint_root': CHECKPOINT_ROOT,
    'results_root': RESULTS_ROOT,
    'dataset_root': DATASET_ROOT,
    'optimizer_scheduler_factory': create_optimizer_scheduler_casme2,
    'criterion_factory': create_criterion_casme2
}

# Configuration validation and summary
print("\n" + "=" * 60)
print("CASME II KEY FRAME SEQUENCE ViT CONFIGURATION COMPLETE")
print("=" * 60)

print(f"Loss Configuration:")
if USE_FOCAL_LOSS:
    print(f"  Function: Optimized Focal Loss")
    print(f"  Gamma: {FOCAL_LOSS_GAMMA}")
    print(f"  Per-class Alpha: {FOCAL_LOSS_ALPHA_WEIGHTS}")
    print(f"  Alpha Sum: {sum(FOCAL_LOSS_ALPHA_WEIGHTS):.3f}")
else:
    print(f"  Function: CrossEntropy with Optimized Weights")
    print(f"  Class Weights: {CROSSENTROPY_CLASS_WEIGHTS}")

print(f"\nModel Configuration:")
print(f"  Architecture: {VIT_MODEL_NAME}")
print(f"  Patch Size: {PATCH_SIZE}px")
print(f"  Input Resolution: 224px (native)")
print(f"  Expected Tokens: {(224 // PATCH_SIZE) ** 2}")
print(f"  Classification Head: 768 -> 256 -> 7 (simplified)")

print(f"\nDataset Configuration:")
print(f"  Version: {CASME2_VIT_CONFIG['dataset_version']}")
print(f"  Classes: {len(CASME2_CLASSES)}")
print(f"  Frame strategy: {CASME2_VIT_CONFIG['frame_strategy']}")
print(f"  Frame types: {CASME2_VIT_CONFIG['frame_types']}")
print(f"  Training approach: {CASME2_VIT_CONFIG['training_approach']}")
print(f"  Inference strategy: {CASME2_VIT_CONFIG['inference_strategy']}")
print(f"  Weight Optimization: {'Per-class Alpha' if USE_FOCAL_LOSS else 'Inverse Sqrt Frequency'}")

print(f"\nTraining Configuration:")
print(f"  Train samples: {preprocessing_info['splits']['train']['total_images']} frames")
print(f"  Validation samples: {preprocessing_info['splits']['val']['total_images']} frames")
print(f"  Test samples: {preprocessing_info['splits']['test']['total_images']} frames")
print(f"  Batch size: {BATCH_SIZE}")
print(f"  Learning rate: {CASME2_VIT_CONFIG['learning_rate']}")
print(f"  Dropout rate: {CASME2_VIT_CONFIG['dropout_rate']}")

print("\nNext: Cell 2 - Dataset Loading and Training Pipeline")

CASME II KEY FRAME SEQUENCE ViT WITH FACE-AWARE PREPROCESSING

[1] Mounting Google Drive...
Mounted at /content/drive
Google Drive mounted successfully

[2] Importing required libraries...
CASME II Key Frame Sequence ViT - Face-Aware Preprocessing Infrastructure
Loading CASME II v8 preprocessing metadata...
Dataset variant: KFS
Processing date: 2025-10-19T08:17:23.905019
Preprocessing method: face_bbox_expansion_all_directions
Total images processed: 765
Face detection rate: 100.00%
Target size: 224x224px
BBox expansion: 20px (all directions)

Dataset split information:
  Train samples: 603
  Validation samples: 78
  Test samples: 84
Using ViT-Base Patch16 for fine-grained micro-expression analysis

EXPERIMENT CONFIGURATION - KEY FRAME SEQUENCE FACE-AWARE
Dataset: v8 Key Frame Sequence with Face-Aware Preprocessing
Frame strategy: onset, apex, offset (3 frames per video)
Training approach: Frame-level independent learning for late fusion
Loss Function: Focal Loss
  Gamma: 2.0
  Alpha W

preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

ViT Image Processor configured for 224px native resolution

Dataset paths:
Train: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/datasets/processed_casme2/preprocessed_v8/train
Validation: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/datasets/processed_casme2/preprocessed_v8/val
Test: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/datasets/processed_casme2/preprocessed_v8/test

ViT CASME II architecture validation...


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ViT feature dimension: 768
ViT CASME II Simplified: 768 -> 256 -> 7
Dropout rate: 0.3 (increased for regularization)
Validation successful: Output shape torch.Size([1, 7])
Expected tokens for 224px with patch16: 196 tokens

CASME II KEY FRAME SEQUENCE ViT CONFIGURATION COMPLETE
Loss Configuration:
  Function: Optimized Focal Loss
  Gamma: 2.0
  Per-class Alpha: [0.055, 0.069, 0.098, 0.107, 0.109, 0.219, 0.343]
  Alpha Sum: 1.000

Model Configuration:
  Architecture: google/vit-base-patch16-224-in21k
  Patch Size: 16px
  Input Resolution: 224px (native)
  Expected Tokens: 196
  Classification Head: 768 -> 256 -> 7 (simplified)

Dataset Configuration:
  Version: v8
  Classes: 7
  Frame strategy: key_frame_sequence
  Frame types: ['onset', 'apex', 'offset']
  Training approach: frame_level_independent
  Inference strategy: late_fusion
  Weight Optimization: Per-class Alpha

Training Configuration:
  Train samples: 603 frames
  Validation samples: 78 frames
  Test samples: 84 frames
  Batc

In [2]:
# @title Cell 2: CASME II Key Frame Sequence ViT Training Pipeline

# File: 06_01_ViT_CASME2_KFS_Cell2.py
# Location: experiments/06_01_ViT_CASME2-KFS-PREP.ipynb
# Purpose: Enhanced training pipeline for CASME II Key Frame Sequence ViT with hardened checkpoint system

import os
import time
import json
import numpy as np
from tqdm import tqdm
from sklearn.metrics import f1_score, precision_recall_fscore_support, accuracy_score
from concurrent.futures import ThreadPoolExecutor
import multiprocessing as mp
import shutil
import tempfile

print("CASME II Key Frame Sequence ViT Training Pipeline")
print("=" * 70)
print(f"Loss Function: {'Optimized Focal Loss' if CASME2_VIT_CONFIG['use_focal_loss'] else 'CrossEntropy Loss'}")
if CASME2_VIT_CONFIG['use_focal_loss']:
    print(f"Focal Loss Parameters:")
    print(f"  Gamma: {CASME2_VIT_CONFIG['focal_loss_gamma']}")
    print(f"  Per-class Alpha: {CASME2_VIT_CONFIG['focal_loss_alpha_weights']}")
    print(f"  Alpha Sum: {sum(CASME2_VIT_CONFIG['focal_loss_alpha_weights']):.3f}")
else:
    print(f"CrossEntropy Parameters:")
    print(f"  Optimized Class Weights: {CASME2_VIT_CONFIG['crossentropy_class_weights']}")
print(f"Dataset version: {CASME2_VIT_CONFIG['dataset_version']}")
print(f"Frame strategy: {CASME2_VIT_CONFIG['frame_strategy']}")
print(f"Training approach: {CASME2_VIT_CONFIG['training_approach']}")
print(f"Training epochs: {CASME2_VIT_CONFIG['num_epochs']}")
print(f"Scheduler patience: {CASME2_VIT_CONFIG['scheduler_patience']}")

# Enhanced CASME II Dataset with optimized RAM caching
class CASME2DatasetTraining(Dataset):
    """Enhanced CASME II dataset for training with RAM caching optimization"""

    def __init__(self, dataset_root, split, transform=None, use_ram_cache=True):
        self.dataset_root = dataset_root
        self.split = split
        self.transform = transform
        self.use_ram_cache = use_ram_cache
        self.images = []
        self.labels = []
        self.filenames = []
        self.cached_images = []

        split_path = os.path.join(dataset_root, split)

        print(f"Loading CASME II {split} dataset for training...")

        if not os.path.exists(split_path):
            raise FileNotFoundError(f"Split directory not found: {split_path}")

        all_files = [f for f in os.listdir(split_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
        print(f"Found {len(all_files)} image files in directory")

        if len(all_files) > 0:
            print(f"Sample filename: {all_files[0]}")

        loaded_count = 0
        skipped_count = 0

        for filename in sorted(all_files):
            # Extract emotion from filename - flexible approach
            emotion_found = None

            # Remove file extension first
            name_without_ext = filename.rsplit('.', 1)[0]

            # Check if any emotion class appears in filename
            for emotion_class in CASME2_CLASSES:
                if emotion_class in name_without_ext.lower():
                    emotion_found = emotion_class
                    break

            if emotion_found and emotion_found in CLASS_TO_IDX:
                image_path = os.path.join(split_path, filename)
                self.images.append(image_path)
                self.labels.append(CLASS_TO_IDX[emotion_found])
                self.filenames.append(filename)
                loaded_count += 1
            else:
                skipped_count += 1
                if skipped_count <= 3:
                    print(f"  Skipped (no emotion found): {filename}")

        print(f"Loaded {len(self.images)} CASME II {split} samples")
        if skipped_count > 0:
            print(f"  Skipped {skipped_count} files (no recognizable emotion)")

        if len(self.images) == 0:
            print(f"ERROR: No samples loaded! Check filename format and emotion labels.")
            print(f"Expected emotions in filenames: {CASME2_CLASSES}")

        self._print_distribution()

        if self.use_ram_cache and len(self.images) > 0:
            self._preload_to_ram()

    def _print_distribution(self):
        """Print class distribution"""
        if len(self.labels) == 0:
            print("  No samples to display distribution")
            return

        label_counts = {}
        for label in self.labels:
            label_counts[label] = label_counts.get(label, 0) + 1

        for label, count in sorted(label_counts.items()):
            class_name = CASME2_CLASSES[label]
            percentage = (count / len(self.labels)) * 100
            print(f"  {class_name}: {count} samples ({percentage:.1f}%)")

    def _preload_to_ram(self):
        """RAM preloading with parallel loading for training efficiency"""
        if len(self.images) == 0:
            print(f"Skipping RAM preload: No images to load")
            return

        print(f"Preloading {len(self.images)} {self.split} images to RAM with {RAM_PRELOAD_WORKERS} workers...")

        self.cached_images = [None] * len(self.images)
        valid_images = 0

        def load_single_image(idx, img_path):
            """Load single image with error handling"""
            try:
                image = Image.open(img_path).convert('RGB')
                if image.size != (224, 224):
                    image = image.resize((224, 224), Image.Resampling.LANCZOS)
                return idx, image, True
            except Exception as e:
                return idx, Image.new('RGB', (224, 224), (128, 128, 128)), False

        with ThreadPoolExecutor(max_workers=RAM_PRELOAD_WORKERS) as executor:
            futures = [executor.submit(load_single_image, i, path)
                      for i, path in enumerate(self.images)]

            for future in tqdm(futures, desc=f"Loading {self.split} to RAM"):
                idx, image, success = future.result()
                self.cached_images[idx] = image
                if success:
                    valid_images += 1

        ram_usage_gb = len(self.cached_images) * 224 * 224 * 3 * 4 / 1e9
        print(f"{self.split.upper()} RAM caching completed: {valid_images}/{len(self.images)} images, ~{ram_usage_gb:.2f}GB")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if self.use_ram_cache and self.cached_images[idx] is not None:
            image = self.cached_images[idx].copy()
        else:
            try:
                image = Image.open(self.images[idx]).convert('RGB')
                if image.size != (224, 224):
                    image = image.resize((224, 224), Image.Resampling.LANCZOS)
            except:
                image = Image.new('RGB', (224, 224), (128, 128, 128))

        if self.transform:
            image = self.transform(image)

        return image, self.labels[idx], self.filenames[idx]

# Enhanced metrics calculation with comprehensive error handling
def calculate_metrics_safe_robust(predictions, labels, class_names, average='macro'):
    """
    Calculate metrics with enhanced error handling and validation

    Args:
        predictions: numpy array or list of predicted class indices (NOT logits)
        labels: numpy array or list of true class indices
        class_names: list of class names
        average: averaging method for metrics
    """
    try:
        # Convert to numpy arrays if needed
        if isinstance(predictions, torch.Tensor):
            predictions = predictions.cpu().numpy()
        else:
            predictions = np.array(predictions)

        if isinstance(labels, torch.Tensor):
            labels = labels.cpu().numpy()
        else:
            labels = np.array(labels)

        # Validate shapes
        if predictions.shape[0] != labels.shape[0]:
            raise ValueError(f"Size mismatch: predictions {predictions.shape[0]} vs labels {labels.shape[0]}")

        # Ensure predictions are 1D (class indices, not one-hot)
        if len(predictions.shape) > 1:
            if predictions.shape[1] > 1:
                predictions = np.argmax(predictions, axis=1)
            else:
                predictions = predictions.flatten()

        # Calculate metrics
        accuracy = accuracy_score(labels, predictions)
        precision, recall, f1, _ = precision_recall_fscore_support(
            labels, predictions,
            average=average,
            zero_division=0,
            labels=list(range(len(class_names)))
        )

        return {
            'accuracy': float(accuracy),
            'precision': float(precision),
            'recall': float(recall),
            'f1_score': float(f1)
        }
    except Exception as e:
        print(f"Warning: Metrics calculation error: {e}")
        import traceback
        traceback.print_exc()
        return {
            'accuracy': 0.0,
            'precision': 0.0,
            'recall': 0.0,
            'f1_score': 0.0
        }

# Enhanced training function with comprehensive error handling
def train_epoch(model, dataloader, criterion, optimizer, device, epoch, total_epochs):
    """Enhanced training epoch with robust error handling"""
    model.train()

    running_loss = 0.0
    all_predictions = []
    all_labels = []

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{total_epochs} [Train]")

    for batch_idx, (images, labels, filenames) in enumerate(progress_bar):
        try:
            images = images.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(images)

            if torch.isnan(outputs).any() or torch.isinf(outputs).any():
                print(f"Warning: Invalid outputs detected at batch {batch_idx}")
                continue

            loss = criterion(outputs, labels)

            if torch.isnan(loss) or torch.isinf(loss):
                print(f"Warning: Invalid loss detected at batch {batch_idx}")
                continue

            loss.backward()

            torch.nn.utils.clip_grad_norm_(model.parameters(), CASME2_VIT_CONFIG['gradient_clip'])

            optimizer.step()

            running_loss += loss.item()

            predictions = torch.argmax(outputs, dim=1)
            all_predictions.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

            progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})

        except Exception as e:
            print(f"Error in training batch {batch_idx}: {e}")
            continue

    epoch_loss = running_loss / len(dataloader)

    # Convert to numpy arrays for metrics calculation
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)

    # Calculate metrics - predictions are already class indices
    metrics = calculate_metrics_safe_robust(
        all_predictions,
        all_labels,
        CASME2_CLASSES,
        average='macro'
    )

    return epoch_loss, metrics

# Enhanced validation function
def validate_epoch(model, dataloader, criterion, device, epoch, total_epochs):
    """Enhanced validation epoch with comprehensive metrics"""
    model.eval()

    running_loss = 0.0
    all_predictions = []
    all_labels = []
    all_filenames = []

    progress_bar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{total_epochs} [Val]")

    with torch.no_grad():
        for batch_idx, (images, labels, filenames) in enumerate(progress_bar):
            try:
                images = images.to(device)
                labels = labels.to(device)

                outputs = model(images)

                if torch.isnan(outputs).any() or torch.isinf(outputs).any():
                    print(f"Warning: Invalid outputs in validation batch {batch_idx}")
                    continue

                loss = criterion(outputs, labels)

                if torch.isnan(loss) or torch.isinf(loss):
                    print(f"Warning: Invalid loss in validation batch {batch_idx}")
                    continue

                running_loss += loss.item()

                predictions = torch.argmax(outputs, dim=1)
                all_predictions.extend(predictions.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())
                all_filenames.extend(filenames)

                progress_bar.set_postfix({'loss': f"{loss.item():.4f}"})

            except Exception as e:
                print(f"Error in validation batch {batch_idx}: {e}")
                continue

    epoch_loss = running_loss / len(dataloader)

    # Convert to numpy arrays for metrics calculation
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)

    # Calculate metrics - predictions are already class indices
    metrics = calculate_metrics_safe_robust(
        all_predictions,
        all_labels,
        CASME2_CLASSES,
        average='macro'
    )

    return epoch_loss, metrics, all_filenames

# Hardened checkpoint saving with atomic writes
def save_checkpoint_robust(model, optimizer, scheduler, epoch, train_metrics, val_metrics,
                          checkpoint_root, best_metrics, config):
    """Hardened checkpoint saving with atomic writes and validation"""
    try:
        checkpoint_data = {
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'scheduler_state_dict': scheduler.state_dict() if scheduler else None,
            'train_metrics': train_metrics,
            'val_metrics': val_metrics,
            'best_metrics': best_metrics,
            'config': config
        }

        temp_path = os.path.join(checkpoint_root, 'checkpoint_temp.pth')
        final_path = os.path.join(checkpoint_root, 'casme2_vit_kfs_best_f1.pth')

        torch.save(checkpoint_data, temp_path)

        test_load = torch.load(temp_path, map_location='cpu')
        if 'model_state_dict' not in test_load:
            raise ValueError("Checkpoint validation failed: missing model_state_dict")

        shutil.move(temp_path, final_path)

        return final_path

    except Exception as e:
        print(f"Error saving checkpoint: {e}")
        if os.path.exists(temp_path):
            os.remove(temp_path)
        return None

# Safe JSON serialization helper
def safe_json_serialize(obj):
    """Convert numpy/torch types to native Python types for JSON serialization"""
    if isinstance(obj, dict):
        return {k: safe_json_serialize(v) for k, v in obj.items()}
    elif isinstance(obj, list):
        return [safe_json_serialize(item) for item in obj]
    elif isinstance(obj, (np.integer, np.int64, np.int32)):
        return int(obj)
    elif isinstance(obj, (np.floating, np.float64, np.float32)):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif torch.is_tensor(obj):
        return obj.cpu().numpy().tolist()
    else:
        return obj

# Create enhanced datasets with RAM caching
print("\nCreating CASME II training and validation datasets...")

train_dataset = CASME2DatasetTraining(
    dataset_root=GLOBAL_CONFIG_CASME2['dataset_root'],
    split='train',
    transform=GLOBAL_CONFIG_CASME2['transform_train'],
    use_ram_cache=True
)

val_dataset = CASME2DatasetTraining(
    dataset_root=GLOBAL_CONFIG_CASME2['dataset_root'],
    split='val',
    transform=GLOBAL_CONFIG_CASME2['transform_val'],
    use_ram_cache=True
)

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=CASME2_VIT_CONFIG['batch_size'],
    shuffle=True,
    num_workers=CASME2_VIT_CONFIG['num_workers'],
    pin_memory=True,
    drop_last=False
)

val_loader = DataLoader(
    val_dataset,
    batch_size=CASME2_VIT_CONFIG['batch_size'],
    shuffle=False,
    num_workers=CASME2_VIT_CONFIG['num_workers'],
    pin_memory=True
)

print(f"\nDataLoader configuration:")
print(f"  Train batches per epoch: {len(train_loader)}")
print(f"  Validation batches per epoch: {len(val_loader)}")
print(f"  Batch size: {CASME2_VIT_CONFIG['batch_size']}")
print(f"  Workers: {CASME2_VIT_CONFIG['num_workers']}")

# Initialize model, optimizer, criterion
print("\nInitializing ViT model for CASME II Key Frame Sequence...")

model = ViTCASME2Baseline(
    num_classes=CASME2_VIT_CONFIG['num_classes'],
    dropout_rate=CASME2_VIT_CONFIG['dropout_rate']
).to(GLOBAL_CONFIG_CASME2['device'])

optimizer, scheduler = GLOBAL_CONFIG_CASME2['optimizer_scheduler_factory'](
    model, CASME2_VIT_CONFIG
)

criterion = GLOBAL_CONFIG_CASME2['criterion_factory'](
    CASME2_VIT_CONFIG['class_weights'],
    CASME2_VIT_CONFIG['use_focal_loss'],
    CASME2_VIT_CONFIG['focal_loss_alpha_weights'],
    CASME2_VIT_CONFIG['focal_loss_gamma']
)

print(f"Model initialized on {GLOBAL_CONFIG_CASME2['device']}")
print(f"Optimizer: AdamW with LR={CASME2_VIT_CONFIG['learning_rate']}")
print(f"Scheduler: ReduceLROnPlateau with patience={CASME2_VIT_CONFIG['scheduler_patience']}")

# Training history tracking
training_history = {
    'train_loss': [],
    'val_loss': [],
    'train_f1': [],
    'val_f1': [],
    'train_acc': [],
    'val_acc': [],
    'learning_rate': [],
    'epoch_time': []
}

# Best metrics tracking
best_metrics = {
    'f1': 0.0,
    'loss': float('inf'),
    'accuracy': 0.0,
    'epoch': 0
}

print("\nStarting CASME II Key Frame Sequence ViT training...")
print(f"Training configuration: {CASME2_VIT_CONFIG['num_epochs']} epochs")
print("=" * 70)

# Main training loop with hardened checkpoint system
start_time = time.time()

for epoch in range(CASME2_VIT_CONFIG['num_epochs']):
    epoch_start_time = time.time()
    print(f"\nEpoch {epoch+1}/{CASME2_VIT_CONFIG['num_epochs']}")

    # Training phase
    train_loss, train_metrics = train_epoch(
        model, train_loader, criterion, optimizer,
        GLOBAL_CONFIG_CASME2['device'], epoch, CASME2_VIT_CONFIG['num_epochs']
    )

    # Validation phase
    val_loss, val_metrics, val_filenames = validate_epoch(
        model, val_loader, criterion,
        GLOBAL_CONFIG_CASME2['device'], epoch, CASME2_VIT_CONFIG['num_epochs']
    )

    # Update scheduler
    if scheduler:
        scheduler.step(val_metrics['f1_score'])

    # Record training history
    epoch_time = time.time() - epoch_start_time
    current_lr = optimizer.param_groups[0]['lr']

    training_history['train_loss'].append(float(train_loss))
    training_history['val_loss'].append(float(val_loss))
    training_history['train_f1'].append(float(train_metrics['f1_score']))
    training_history['val_f1'].append(float(val_metrics['f1_score']))
    training_history['train_acc'].append(float(train_metrics['accuracy']))
    training_history['val_acc'].append(float(val_metrics['accuracy']))
    training_history['learning_rate'].append(float(current_lr))
    training_history['epoch_time'].append(float(epoch_time))

    # Print epoch summary
    print(f"Train - Loss: {train_loss:.4f}, F1: {train_metrics['f1_score']:.4f}, Acc: {train_metrics['accuracy']:.4f}")
    print(f"Val   - Loss: {val_loss:.4f}, F1: {val_metrics['f1_score']:.4f}, Acc: {val_metrics['accuracy']:.4f}")
    print(f"Time  - Epoch: {epoch_time:.1f}s, LR: {current_lr:.2e}")

    # Enhanced multi-criteria checkpoint saving logic
    save_model = False
    improvement_reason = ""

    if val_metrics['f1_score'] > best_metrics['f1']:
        save_model = True
        improvement_reason = "Higher F1"
    elif val_metrics['f1_score'] == best_metrics['f1']:
        if val_loss < best_metrics['loss']:
            save_model = True
            improvement_reason = "Same F1, Lower Loss"
        elif val_loss == best_metrics['loss'] and val_metrics['accuracy'] > best_metrics['accuracy']:
            save_model = True
            improvement_reason = "Same F1&Loss, Higher Accuracy"

    if save_model:
        best_metrics['f1'] = val_metrics['f1_score']
        best_metrics['loss'] = val_loss
        best_metrics['accuracy'] = val_metrics['accuracy']
        best_metrics['epoch'] = epoch + 1

        best_model_path = save_checkpoint_robust(
            model, optimizer, scheduler, epoch,
            train_metrics, val_metrics, GLOBAL_CONFIG_CASME2['checkpoint_root'],
            best_metrics, CASME2_VIT_CONFIG
        )

        if best_model_path:
            print(f"New best model: {improvement_reason} - F1: {best_metrics['f1']:.4f}")
        else:
            print(f"Warning: Failed to save checkpoint despite improvement")

    # Progress tracking
    elapsed_time = time.time() - start_time
    estimated_total = (elapsed_time / (epoch + 1)) * CASME2_VIT_CONFIG['num_epochs']
    remaining_time = estimated_total - elapsed_time
    progress_pct = ((epoch + 1) / CASME2_VIT_CONFIG['num_epochs']) * 100

    print(f"Progress: {progress_pct:.1f}% | Best F1: {best_metrics['f1']:.4f} | ETA: {remaining_time/60:.1f}min")

# Training completion
total_time = time.time() - start_time
actual_epochs = CASME2_VIT_CONFIG['num_epochs']

print("\n" + "=" * 70)
print("CASME II KEY FRAME SEQUENCE ViT TRAINING COMPLETED")
print("=" * 70)
print(f"Training time: {total_time/60:.1f} minutes")
print(f"Epochs completed: {actual_epochs}")
print(f"Best validation F1: {best_metrics['f1']:.4f} (epoch {best_metrics['epoch']})")
print(f"Final train F1: {training_history['train_f1'][-1]:.4f}")
print(f"Final validation F1: {training_history['val_f1'][-1]:.4f}")

# Enhanced training documentation export
results_dir = GLOBAL_CONFIG_CASME2['results_root']
os.makedirs(f"{results_dir}/training_logs", exist_ok=True)

training_history_path = f"{results_dir}/training_logs/casme2_vit_kfs_training_history.json"

print("\nExporting enhanced training documentation...")

try:
    training_summary = {
        'experiment_type': 'CASME2_ViT_KeyFrameSequence',
        'experiment_configuration': {
            'dataset_version': CASME2_VIT_CONFIG['dataset_version'],
            'frame_strategy': CASME2_VIT_CONFIG['frame_strategy'],
            'frame_types': CASME2_VIT_CONFIG['frame_types'],
            'training_approach': CASME2_VIT_CONFIG['training_approach'],
            'inference_strategy': CASME2_VIT_CONFIG['inference_strategy'],
            'loss_function': 'Optimized Focal Loss' if CASME2_VIT_CONFIG['use_focal_loss'] else 'CrossEntropy',
            'weight_approach': 'Per-class Alpha (sum=1.0)' if CASME2_VIT_CONFIG['use_focal_loss'] else 'Inverse Sqrt Frequency',
            'focal_loss_gamma': CASME2_VIT_CONFIG['focal_loss_gamma'],
            'focal_loss_alpha_weights': CASME2_VIT_CONFIG['focal_loss_alpha_weights'],
            'crossentropy_class_weights': CASME2_VIT_CONFIG['crossentropy_class_weights'],
            'vit_model': CASME2_VIT_CONFIG['vit_model'],
            'patch_size': CASME2_VIT_CONFIG['patch_size']
        },
        'training_history': safe_json_serialize(training_history),
        'best_val_f1': float(best_metrics['f1']),
        'best_val_loss': float(best_metrics['loss']),
        'best_val_accuracy': float(best_metrics['accuracy']),
        'best_epoch': int(best_metrics['epoch']),
        'total_epochs': int(actual_epochs),
        'total_time_minutes': float(total_time / 60),
        'average_epoch_time_seconds': float(np.mean(training_history['epoch_time'])),
        'config': safe_json_serialize(CASME2_VIT_CONFIG),
        'final_train_f1': float(training_history['train_f1'][-1]),
        'final_val_f1': float(training_history['val_f1'][-1]),
        'model_checkpoint': 'casme2_vit_kfs_best_f1.pth',
        'dataset_info': {
            'name': 'CASME_II',
            'version': CASME2_VIT_CONFIG['dataset_version'],
            'frame_strategy': CASME2_VIT_CONFIG['frame_strategy'],
            'frame_types': CASME2_VIT_CONFIG['frame_types'],
            'train_samples': len(train_dataset),
            'val_samples': len(val_dataset),
            'num_classes': 7,
            'class_names': CASME2_CLASSES
        },
        'architecture_info': {
            'model_type': 'ViTCASME2Baseline',
            'backbone': CASME2_VIT_CONFIG['vit_model'],
            'input_size': f"{CASME2_VIT_CONFIG['input_size']}x{CASME2_VIT_CONFIG['input_size']}",
            'patch_size': CASME2_VIT_CONFIG['patch_size'],
            'expected_tokens': (CASME2_VIT_CONFIG['input_size'] // CASME2_VIT_CONFIG['patch_size']) ** 2,
            'classification_head': '768->256->7'
        },
        'enhanced_features': {
            'hardened_checkpoint_system': True,
            'atomic_checkpoint_save': True,
            'checkpoint_validation': True,
            'model_output_validation': True,
            'enhanced_error_handling': True,
            'multi_criteria_checkpoint_logic': True,
            'memory_optimized_training': True,
            'ram_caching': True
        }
    }

    with open(training_history_path, 'w') as f:
        json.dump(training_summary, f, indent=2)

    print(f"Enhanced training documentation saved successfully: {training_history_path}")
    print(f"Experiment details: {training_summary['experiment_configuration']['loss_function']} loss")
    if CASME2_VIT_CONFIG['use_focal_loss']:
        print(f"  Gamma: {CASME2_VIT_CONFIG['focal_loss_gamma']}, Alpha Sum: {sum(CASME2_VIT_CONFIG['focal_loss_alpha_weights']):.3f}")
    print(f"Model variant: {CASME2_VIT_CONFIG['vit_model']}")
    print(f"Dataset version: {CASME2_VIT_CONFIG['dataset_version']}")

except Exception as e:
    print(f"Warning: Could not save training documentation: {e}")
    print("Training completed successfully, but documentation export failed")

# Enhanced memory cleanup
if torch.cuda.is_available():
    torch.cuda.synchronize()
    torch.cuda.empty_cache()

print("\nNext: Cell 3 - CASME II Key Frame Sequence ViT Evaluation with Late Fusion")
print("Enhanced training pipeline with hardened checkpoint system completed successfully!")

CASME II Key Frame Sequence ViT Training Pipeline
Loss Function: Optimized Focal Loss
Focal Loss Parameters:
  Gamma: 2.0
  Per-class Alpha: [0.055, 0.069, 0.098, 0.107, 0.109, 0.219, 0.343]
  Alpha Sum: 1.000
Dataset version: v8
Frame strategy: key_frame_sequence
Training approach: frame_level_independent
Training epochs: 50
Scheduler patience: 5

Creating CASME II training and validation datasets...
Loading CASME II train dataset for training...
Found 603 image files in directory
Sample filename: sub01_EP02_01f_onset_happiness.jpg
Loaded 603 CASME II train samples
  others: 237 samples (39.3%)
  disgust: 150 samples (24.9%)
  happiness: 75 samples (12.4%)
  repression: 63 samples (10.4%)
  surprise: 60 samples (10.0%)
  sadness: 15 samples (2.5%)
  fear: 3 samples (0.5%)
Preloading 603 train images to RAM with 32 workers...


Loading train to RAM: 100%|██████████| 603/603 [00:08<00:00, 69.11it/s] 


TRAIN RAM caching completed: 603/603 images, ~0.36GB
Loading CASME II val dataset for training...
Found 78 image files in directory
Sample filename: sub01_EP03_02_apex_others.jpg
Loaded 78 CASME II val samples
  others: 30 samples (38.5%)
  disgust: 18 samples (23.1%)
  happiness: 9 samples (11.5%)
  repression: 9 samples (11.5%)
  surprise: 6 samples (7.7%)
  sadness: 3 samples (3.8%)
  fear: 3 samples (3.8%)
Preloading 78 val images to RAM with 32 workers...


Loading val to RAM: 100%|██████████| 78/78 [00:01<00:00, 47.93it/s]


VAL RAM caching completed: 78/78 images, ~0.05GB

DataLoader configuration:
  Train batches per epoch: 76
  Validation batches per epoch: 10
  Batch size: 8
  Workers: 4

Initializing ViT model for CASME II Key Frame Sequence...


...

Train - Loss: 0.0001, F1: 1.0000, Acc: 1.0000
Val   - Loss: 0.2254, F1: 0.1313, Acc: 0.3077
Time  - Epoch: 12.1s, LR: 1.00e-06
Progress: 96.0% | Best F1: 0.1691 | ETA: 0.4min

Epoch 49/50


Epoch 49/50 [Val]: 100%|██████████| 10/10 [00:00<00:00, 16.88it/s, loss=0.0869]


Train - Loss: 0.0001, F1: 1.0000, Acc: 1.0000
Val   - Loss: 0.2259, F1: 0.1313, Acc: 0.3077
Time  - Epoch: 12.2s, LR: 1.00e-06
Progress: 98.0% | Best F1: 0.1691 | ETA: 0.2min

Epoch 50/50


Epoch 50/50 [Val]: 100%|██████████| 10/10 [00:00<00:00, 17.03it/s, loss=0.0865]

Train - Loss: 0.0001, F1: 1.0000, Acc: 1.0000
Val   - Loss: 0.2263, F1: 0.1328, Acc: 0.3077
Time  - Epoch: 12.2s, LR: 1.00e-06
Progress: 100.0% | Best F1: 0.1691 | ETA: 0.0min

CASME II KEY FRAME SEQUENCE ViT TRAINING COMPLETED
Training time: 10.2 minutes
Epochs completed: 50
Best validation F1: 0.1691 (epoch 1)
Final train F1: 1.0000
Final validation F1: 0.1328

Exporting enhanced training documentation...
Training completed successfully, but documentation export failed

Next: Cell 3 - CASME II Key Frame Sequence ViT Evaluation with Late Fusion
Enhanced training pipeline with hardened checkpoint system completed successfully!


In [3]:
# @title Cell 3: CASME II ViT Evaluation with Dual Dataset Support

# File: 06_01_ViT_CASME2_KFS_Cell3.py
# Location: experiments/06_01_ViT_CASME2-KFS-PREP.ipynb
# Purpose: Comprehensive evaluation framework with support for AF (v7) and KFS (v8) with late fusion

import os
import time
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from collections import defaultdict

from sklearn.metrics import (
    accuracy_score, precision_recall_fscore_support,
    classification_report, confusion_matrix,
    roc_curve, auc
)
from sklearn.preprocessing import label_binarize
from concurrent.futures import ThreadPoolExecutor
import warnings
warnings.filterwarnings('ignore')

# =====================================================
# DUAL DATASET EVALUATION CONFIGURATION
# =====================================================
# Configure which test datasets to evaluate:
# 'v7' = Apex Frame preprocessing (28 samples, frame-level evaluation)
# 'v8' = Key Frame Sequence preprocessing (84 frames -> 28 videos with late fusion)

EVALUATE_DATASETS = ['v7', 'v8']  # Can be ['v7'], ['v8'], or ['v7', 'v8']

print("CASME II ViT Evaluation Framework with Dual Dataset Support")
print("=" * 60)
print(f"Datasets to evaluate: {EVALUATE_DATASETS}")
print("=" * 60)

# =====================================================
# DATASET CONFIGURATION FUNCTION
# =====================================================

def get_test_dataset_config(version, project_root):
    """
    Get test dataset configuration based on version

    Args:
        version: 'v7' (AF) or 'v8' (KFS)
        project_root: Project root path

    Returns:
        dict: Configuration for selected test dataset
    """
    if version == 'v7':
        config = {
            'version': 'v7',
            'variant': 'AF',
            'dataset_path': f"{project_root}/datasets/processed_casme2/preprocessed_v7",
            'preprocessing_summary': 'preprocessing_summary.json',
            'description': 'Apex Frame with Face-Aware Preprocessing',
            'expected_samples': 28,
            'frame_strategy': 'apex_frame',
            'evaluation_mode': 'frame_level',
            'aggregation': None
        }
    elif version == 'v8':
        config = {
            'version': 'v8',
            'variant': 'KFS',
            'dataset_path': f"{project_root}/datasets/processed_casme2/preprocessed_v8",
            'preprocessing_summary': 'preprocessing_summary.json',
            'description': 'Key Frame Sequence with Face-Aware Preprocessing',
            'expected_frames': 84,
            'expected_videos': 28,
            'frame_strategy': 'key_frame_sequence',
            'frame_types': ['onset', 'apex', 'offset'],
            'evaluation_mode': 'video_level',
            'aggregation': 'late_fusion'
        }
    else:
        raise ValueError(f"Invalid version: {version}. Must be 'v7' or 'v8'")

    return config

# =====================================================
# VIDEO ID EXTRACTION FOR KFS LATE FUSION
# =====================================================

def extract_video_id_from_filename(filename):
    """
    Extract video ID from KFS filename by removing frame type suffix

    Expected format: sub01_EP02_01f_happiness_onset.jpg
    Video ID: sub01_EP02_01f_happiness

    Args:
        filename: Image filename with frame type

    Returns:
        str: Video ID without frame type
    """
    # Remove file extension
    name_without_ext = filename.rsplit('.', 1)[0]

    # Remove frame type suffix (onset, apex, offset)
    for frame_type in ['onset', 'apex', 'offset']:
        if name_without_ext.endswith(f'_{frame_type}'):
            video_id = name_without_ext.rsplit(f'_{frame_type}', 1)[0]
            return video_id

    # If no frame type found, return as is
    return name_without_ext

# =====================================================
# ENHANCED TEST DATASET CLASS
# =====================================================

class CASME2DatasetEvaluation(Dataset):
    """Enhanced CASME II test dataset with comprehensive evaluation support"""

    def __init__(self, dataset_root, split, transform=None, use_ram_cache=True):
        self.dataset_root = dataset_root
        self.split = split
        self.transform = transform
        self.use_ram_cache = use_ram_cache
        self.images = []
        self.labels = []
        self.filenames = []
        self.emotions = []
        self.video_ids = []
        self.cached_images = []

        split_path = os.path.join(dataset_root, split)

        print(f"Loading CASME II {split} dataset for evaluation...")

        if not os.path.exists(split_path):
            raise FileNotFoundError(f"Split directory not found: {split_path}")

        all_files = [f for f in os.listdir(split_path) if f.endswith(('.jpg', '.png', '.jpeg'))]
        print(f"Found {len(all_files)} image files in directory")

        loaded_count = 0

        for filename in sorted(all_files):
            emotion_found = None
            name_without_ext = filename.rsplit('.', 1)[0]

            for emotion_class in CASME2_CLASSES:
                if emotion_class in name_without_ext.lower():
                    emotion_found = emotion_class
                    break

            if emotion_found and emotion_found in CLASS_TO_IDX:
                image_path = os.path.join(split_path, filename)
                video_id = extract_video_id_from_filename(filename)

                self.images.append(image_path)
                self.labels.append(CLASS_TO_IDX[emotion_found])
                self.filenames.append(filename)
                self.emotions.append(emotion_found)
                self.video_ids.append(video_id)
                loaded_count += 1

        print(f"Loaded {len(self.images)} CASME II {split} samples for evaluation")
        self._print_evaluation_distribution()

        if self.use_ram_cache and len(self.images) > 0:
            self._preload_to_ram_evaluation()

    def _print_evaluation_distribution(self):
        """Print comprehensive distribution for evaluation analysis"""
        if len(self.labels) == 0:
            print("No test samples found!")
            return

        label_counts = {}
        unique_videos = set(self.video_ids)

        for label in self.labels:
            label_counts[label] = label_counts.get(label, 0) + 1

        print("Test set class distribution:")
        for label, count in sorted(label_counts.items()):
            class_name = CASME2_CLASSES[label]
            percentage = (count / len(self.labels)) * 100
            print(f"  {class_name}: {count} samples ({percentage:.1f}%)")

        print(f"Unique video IDs: {len(unique_videos)}")

        missing_classes = []
        for i, class_name in enumerate(CASME2_CLASSES):
            if i not in label_counts:
                missing_classes.append(class_name)

        if missing_classes:
            print(f"Missing classes in test set: {missing_classes}")

    def _preload_to_ram_evaluation(self):
        """RAM preloading optimized for evaluation"""
        if len(self.images) == 0:
            return

        print(f"Preloading {len(self.images)} test images to RAM with {RAM_PRELOAD_WORKERS} workers...")

        self.cached_images = [None] * len(self.images)
        valid_images = 0

        def load_single_image(idx, img_path):
            try:
                image = Image.open(img_path).convert('RGB')
                if image.size != (224, 224):
                    image = image.resize((224, 224), Image.Resampling.LANCZOS)
                return idx, image, True
            except Exception as e:
                return idx, Image.new('RGB', (224, 224), (128, 128, 128)), False

        with ThreadPoolExecutor(max_workers=RAM_PRELOAD_WORKERS) as executor:
            futures = [executor.submit(load_single_image, i, path)
                      for i, path in enumerate(self.images)]

            for future in tqdm(futures, desc="Loading test to RAM"):
                idx, image, success = future.result()
                self.cached_images[idx] = image
                if success:
                    valid_images += 1

        ram_usage_gb = len(self.cached_images) * 224 * 224 * 3 * 4 / 1e9
        print(f"TEST RAM caching completed: {valid_images}/{len(self.images)} images, ~{ram_usage_gb:.2f}GB")

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        if self.use_ram_cache and self.cached_images[idx] is not None:
            image = self.cached_images[idx].copy()
        else:
            try:
                image = Image.open(self.images[idx]).convert('RGB')
                if image.size != (224, 224):
                    image = image.resize((224, 224), Image.Resampling.LANCZOS)
            except:
                image = Image.new('RGB', (224, 224), (128, 128, 128))

        if self.transform:
            image = self.transform(image)

        return image, self.labels[idx], self.filenames[idx], self.video_ids[idx]

# =====================================================
# MODEL LOADING FUNCTION
# =====================================================

def load_trained_model_casme2(checkpoint_path, device):
    """Load trained model from checkpoint"""
    print(f"Loading trained model from: {checkpoint_path}")

    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")

    checkpoint = torch.load(checkpoint_path, map_location=device)

    model = ViTCASME2Baseline(
        num_classes=7,
        dropout_rate=CASME2_VIT_CONFIG['dropout_rate']
    ).to(device)

    model.load_state_dict(checkpoint['model_state_dict'])
    model.eval()

    training_info = {
        'best_epoch': checkpoint.get('epoch', 0),
        'best_val_f1': checkpoint.get('val_metrics', {}).get('f1_score', 0.0),
        'best_val_accuracy': checkpoint.get('val_metrics', {}).get('accuracy', 0.0),
        'config': checkpoint.get('config', {})
    }

    print(f"Model loaded successfully from epoch {training_info['best_epoch']}")
    print(f"Training best val F1: {training_info['best_val_f1']:.4f}")

    return model, training_info

# =====================================================
# FRAME-LEVEL INFERENCE (for AF v7)
# =====================================================

def run_frame_level_inference(model, dataloader, device):
    """
    Run frame-level inference for Apex Frame evaluation
    Returns predictions, probabilities, labels for each frame
    """
    model.eval()

    all_predictions = []
    all_probabilities = []
    all_labels = []
    all_filenames = []
    all_video_ids = []

    print("Running frame-level inference...")

    inference_start = time.time()

    with torch.no_grad():
        for images, labels, filenames, video_ids in tqdm(dataloader, desc="Frame-level inference"):
            images = images.to(device)

            outputs = model(images)
            probabilities = torch.softmax(outputs, dim=1)
            predictions = torch.argmax(probabilities, dim=1)

            all_predictions.extend(predictions.cpu().numpy())
            all_probabilities.extend(probabilities.cpu().numpy())
            all_labels.extend(labels.numpy())
            all_filenames.extend(filenames)
            all_video_ids.extend(video_ids)

    inference_time = time.time() - inference_start

    results = {
        'predictions': np.array(all_predictions),
        'probabilities': np.array(all_probabilities),
        'labels': np.array(all_labels),
        'filenames': all_filenames,
        'video_ids': all_video_ids,
        'inference_time': inference_time,
        'evaluation_mode': 'frame_level'
    }

    print(f"Frame-level inference completed: {len(all_predictions)} predictions in {inference_time:.2f}s")

    return results

# =====================================================
# VIDEO-LEVEL INFERENCE WITH LATE FUSION (for KFS v8)
# =====================================================

def run_video_level_inference_late_fusion(model, dataloader, device):
    """
    Run video-level inference with late fusion for Key Frame Sequence

    Process:
    1. Get frame-level probabilities for all frames (onset, apex, offset)
    2. Group frames by video ID
    3. Average probabilities across frames per video
    4. Get final video-level prediction from averaged probabilities

    Returns video-level predictions (28 videos from 84 frames)
    """
    model.eval()

    # Step 1: Get frame-level probabilities
    frame_data = defaultdict(lambda: {
        'probabilities': [],
        'filenames': [],
        'label': None
    })

    print("Running frame-level inference for late fusion...")

    inference_start = time.time()

    with torch.no_grad():
        for images, labels, filenames, video_ids in tqdm(dataloader, desc="Processing frames"):
            images = images.to(device)

            outputs = model(images)
            probabilities = torch.softmax(outputs, dim=1)

            # Group by video ID
            for i in range(len(video_ids)):
                video_id = video_ids[i]
                prob = probabilities[i].cpu().numpy()
                label = labels[i].item()
                filename = filenames[i]

                frame_data[video_id]['probabilities'].append(prob)
                frame_data[video_id]['filenames'].append(filename)
                frame_data[video_id]['label'] = label

    # Step 2: Late fusion - average probabilities per video
    print(f"Applying late fusion across {len(frame_data)} videos...")

    video_predictions = []
    video_probabilities = []
    video_labels = []
    video_ids = []
    video_frame_counts = []

    for video_id, data in sorted(frame_data.items()):
        # Average probabilities across all frames of this video
        frame_probs = np.array(data['probabilities'])
        avg_prob = np.mean(frame_probs, axis=0)

        # Final prediction from averaged probabilities
        video_pred = np.argmax(avg_prob)

        video_predictions.append(video_pred)
        video_probabilities.append(avg_prob)
        video_labels.append(data['label'])
        video_ids.append(video_id)
        video_frame_counts.append(len(data['probabilities']))

    inference_time = time.time() - inference_start

    results = {
        'predictions': np.array(video_predictions),
        'probabilities': np.array(video_probabilities),
        'labels': np.array(video_labels),
        'video_ids': video_ids,
        'frame_counts': video_frame_counts,
        'frame_data': dict(frame_data),
        'inference_time': inference_time,
        'evaluation_mode': 'video_level_late_fusion',
        'total_frames': sum(video_frame_counts),
        'total_videos': len(video_ids)
    }

    print(f"Late fusion completed: {len(video_ids)} video predictions from {sum(video_frame_counts)} frames")
    print(f"Inference time: {inference_time:.2f}s")

    return results

# =====================================================
# COMPREHENSIVE METRICS CALCULATION
# =====================================================

def calculate_comprehensive_metrics(inference_results):
    """Calculate comprehensive metrics for evaluation"""

    predictions = inference_results['predictions']
    labels = inference_results['labels']
    probabilities = inference_results['probabilities']

    unique_labels = np.unique(labels)
    available_classes = [CASME2_CLASSES[i] for i in unique_labels]
    missing_classes = [cls for i, cls in enumerate(CASME2_CLASSES) if i not in unique_labels]

    # Overall metrics
    accuracy = accuracy_score(labels, predictions)
    precision, recall, f1, support = precision_recall_fscore_support(
        labels, predictions,
        average='macro',
        zero_division=0,
        labels=list(range(len(CASME2_CLASSES)))
    )

    # Per-class metrics
    precision_per_class, recall_per_class, f1_per_class, support_per_class = precision_recall_fscore_support(
        labels, predictions,
        average=None,
        zero_division=0,
        labels=list(range(len(CASME2_CLASSES)))
    )

    # Confusion matrix
    cm = confusion_matrix(labels, predictions, labels=list(range(len(CASME2_CLASSES))))

    # ROC and AUC calculation
    labels_binarized = label_binarize(labels, classes=list(range(len(CASME2_CLASSES))))

    auc_scores = {}
    fpr_dict = {}
    tpr_dict = {}

    for i, class_name in enumerate(CASME2_CLASSES):
        if i in unique_labels and np.sum(labels_binarized[:, i]) > 0:
            fpr, tpr, _ = roc_curve(labels_binarized[:, i], probabilities[:, i])
            auc_score = auc(fpr, tpr)
            auc_scores[class_name] = float(auc_score)
            fpr_dict[class_name] = fpr.tolist()
            tpr_dict[class_name] = tpr.tolist()
        else:
            auc_scores[class_name] = 0.0
            fpr_dict[class_name] = []
            tpr_dict[class_name] = []

    macro_auc = np.mean([score for score in auc_scores.values() if score > 0])

    # Compile results
    evaluation_results = {
        'evaluation_metadata': {
            'dataset': 'CASME_II',
            'model_type': 'ViTCASME2Baseline',
            'evaluation_timestamp': datetime.now().isoformat(),
            'evaluation_mode': inference_results['evaluation_mode'],
            'test_samples': len(predictions),
            'class_names': CASME2_CLASSES,
            'available_classes': available_classes,
            'missing_classes': missing_classes
        },

        'overall_performance': {
            'accuracy': float(accuracy),
            'macro_precision': float(precision),
            'macro_recall': float(recall),
            'macro_f1': float(f1),
            'macro_auc': float(macro_auc)
        },

        'per_class_performance': {},

        'confusion_matrix': cm.tolist(),

        'roc_analysis': {
            'auc_scores': auc_scores,
            'fpr_curves': fpr_dict,
            'tpr_curves': tpr_dict
        },

        'inference_performance': {
            'total_time_seconds': float(inference_results['inference_time']),
            'average_time_ms_per_sample': float(inference_results['inference_time'] * 1000 / len(predictions))
        }
    }

    # Add video-level specific info if applicable
    if 'total_frames' in inference_results:
        evaluation_results['kfs_late_fusion_info'] = {
            'total_frames': inference_results['total_frames'],
            'total_videos': inference_results['total_videos'],
            'aggregation_method': 'probability_averaging',
            'frames_per_video': inference_results['frame_counts']
        }

    # Per-class performance details
    for i, class_name in enumerate(CASME2_CLASSES):
        evaluation_results['per_class_performance'][class_name] = {
            'precision': float(precision_per_class[i]),
            'recall': float(recall_per_class[i]),
            'f1_score': float(f1_per_class[i]),
            'support': int(support_per_class[i]),
            'auc': auc_scores[class_name],
            'in_test_set': i in unique_labels
        }

    return evaluation_results

# =====================================================
# WRONG PREDICTIONS ANALYSIS
# =====================================================

def analyze_wrong_predictions(inference_results):
    """Analyze wrong predictions for error analysis"""

    predictions = inference_results['predictions']
    labels = inference_results['labels']

    if 'filenames' in inference_results:
        identifiers = inference_results['filenames']
    elif 'video_ids' in inference_results:
        identifiers = inference_results['video_ids']
    else:
        identifiers = [f"sample_{i}" for i in range(len(predictions))]

    wrong_predictions = []
    wrong_by_class = defaultdict(list)
    confusion_patterns = defaultdict(int)

    for i in range(len(predictions)):
        if predictions[i] != labels[i]:
            true_class = CASME2_CLASSES[labels[i]]
            pred_class = CASME2_CLASSES[predictions[i]]

            wrong_pred = {
                'identifier': identifiers[i],
                'true_class': true_class,
                'predicted_class': pred_class,
                'true_label': int(labels[i]),
                'predicted_label': int(predictions[i])
            }

            wrong_predictions.append(wrong_pred)
            wrong_by_class[true_class].append(wrong_pred)

            pattern = f"{true_class} -> {pred_class}"
            confusion_patterns[pattern] += 1

    error_summary = {cls: len(wrong_by_class[cls]) for cls in CASME2_CLASSES}

    results = {
        'analysis_metadata': {
            'total_samples': len(predictions),
            'total_wrong_predictions': len(wrong_predictions),
            'overall_error_rate': (len(wrong_predictions) / len(predictions)) * 100
        },
        'wrong_predictions': wrong_predictions,
        'wrong_predictions_by_class': dict(wrong_by_class),
        'error_summary': error_summary,
        'confusion_patterns': dict(confusion_patterns)
    }

    return results

# =====================================================
# SAVE EVALUATION RESULTS
# =====================================================

def save_evaluation_results(evaluation_results, wrong_predictions_results, results_dir, test_version):
    """Save comprehensive evaluation results"""
    os.makedirs(results_dir, exist_ok=True)

    results_file = f"{results_dir}/casme2_vit_evaluation_results_{test_version}.json"
    with open(results_file, 'w') as f:
        json.dump(evaluation_results, f, indent=2, default=str)

    wrong_predictions_file = f"{results_dir}/casme2_vit_wrong_predictions_{test_version}.json"
    with open(wrong_predictions_file, 'w') as f:
        json.dump(wrong_predictions_results, f, indent=2, default=str)

    print(f"Evaluation results saved:")
    print(f"  Main results: {os.path.basename(results_file)}")
    print(f"  Wrong predictions: {os.path.basename(wrong_predictions_file)}")

    return results_file, wrong_predictions_file

# =====================================================
# MAIN EVALUATION EXECUTION
# =====================================================

all_evaluation_results = {}

for dataset_version in EVALUATE_DATASETS:
    print("\n" + "=" * 70)
    print(f"EVALUATING DATASET: {dataset_version.upper()}")
    print("=" * 70)

    try:
        # Get dataset configuration
        test_config = get_test_dataset_config(dataset_version, PROJECT_ROOT)

        print(f"\nTest Dataset Configuration:")
        print(f"  Version: {test_config['version']}")
        print(f"  Variant: {test_config['variant']}")
        print(f"  Description: {test_config['description']}")
        print(f"  Frame strategy: {test_config['frame_strategy']}")
        print(f"  Evaluation mode: {test_config['evaluation_mode']}")
        if 'aggregation' in test_config and test_config['aggregation']:
            print(f"  Aggregation: {test_config['aggregation']}")
        print(f"  Dataset path: {test_config['dataset_path']}")

        # Create test dataset
        print(f"\nCreating CASME II test dataset from {test_config['variant']}...")
        test_dataset = CASME2DatasetEvaluation(
            dataset_root=test_config['dataset_path'],
            split='test',
            transform=GLOBAL_CONFIG_CASME2['transform_val'],
            use_ram_cache=True
        )

        if len(test_dataset) == 0:
            raise ValueError(f"No test samples found for {dataset_version}!")

        test_loader = DataLoader(
            test_dataset,
            batch_size=CASME2_VIT_CONFIG['batch_size'],
            shuffle=False,
            num_workers=CASME2_VIT_CONFIG['num_workers'],
            pin_memory=True
        )

        # Load trained model
        checkpoint_path = f"{GLOBAL_CONFIG_CASME2['checkpoint_root']}/casme2_vit_kfs_best_f1.pth"
        model, training_info = load_trained_model_casme2(checkpoint_path, GLOBAL_CONFIG_CASME2['device'])

        # Run inference based on evaluation mode
        if test_config['evaluation_mode'] == 'frame_level':
            print(f"\nRunning frame-level evaluation for {test_config['variant']}...")
            inference_results = run_frame_level_inference(model, test_loader, GLOBAL_CONFIG_CASME2['device'])

        elif test_config['evaluation_mode'] == 'video_level':
            print(f"\nRunning video-level evaluation with late fusion for {test_config['variant']}...")
            inference_results = run_video_level_inference_late_fusion(model, test_loader, GLOBAL_CONFIG_CASME2['device'])

        else:
            raise ValueError(f"Unknown evaluation mode: {test_config['evaluation_mode']}")

        # Calculate comprehensive metrics
        evaluation_results = calculate_comprehensive_metrics(inference_results)

        # Analyze wrong predictions
        wrong_predictions_results = analyze_wrong_predictions(inference_results)

        # Add training information
        evaluation_results['training_information'] = training_info
        evaluation_results['test_configuration'] = test_config

        # Save results
        results_dir = f"{GLOBAL_CONFIG_CASME2['results_root']}/evaluation_results"
        save_evaluation_results(
            evaluation_results, wrong_predictions_results, results_dir, test_config['version']
        )

        # Store for comparison
        all_evaluation_results[dataset_version] = {
            'evaluation': evaluation_results,
            'wrong_predictions': wrong_predictions_results,
            'config': test_config
        }

        # Display results
        print("\n" + "=" * 60)
        print(f"EVALUATION RESULTS - {test_config['variant']} ({dataset_version})")
        print("=" * 60)

        overall = evaluation_results['overall_performance']
        print(f"\nOverall Performance:")
        print(f"  Accuracy:  {overall['accuracy']:.4f}")
        print(f"  Precision: {overall['macro_precision']:.4f}")
        print(f"  Recall:    {overall['macro_recall']:.4f}")
        print(f"  F1 Score:  {overall['macro_f1']:.4f}")
        print(f"  AUC:       {overall['macro_auc']:.4f}")

        if 'kfs_late_fusion_info' in evaluation_results:
            fusion_info = evaluation_results['kfs_late_fusion_info']
            print(f"\nLate Fusion Info:")
            print(f"  Total frames processed: {fusion_info['total_frames']}")
            print(f"  Video-level predictions: {fusion_info['total_videos']}")
            print(f"  Aggregation method: {fusion_info['aggregation_method']}")

        print(f"\nPer-Class Performance:")
        for class_name, metrics in evaluation_results['per_class_performance'].items():
            in_test = "Present" if metrics['in_test_set'] else "Missing"
            print(f"  {class_name} [{in_test}]: F1={metrics['f1_score']:.4f}, "
                  f"Support={metrics['support']}")

        wrong_meta = wrong_predictions_results['analysis_metadata']
        print(f"\nWrong Predictions Analysis:")
        print(f"  Total errors: {wrong_meta['total_wrong_predictions']} / {wrong_meta['total_samples']}")
        print(f"  Error rate: {wrong_meta['overall_error_rate']:.2f}%")

        print(f"\nInference Performance:")
        print(f"  Total time: {inference_results['inference_time']:.2f}s")
        print(f"  Speed: {evaluation_results['inference_performance']['average_time_ms_per_sample']:.1f} ms/sample")

    except Exception as e:
        print(f"Evaluation failed for {dataset_version}: {e}")
        import traceback
        traceback.print_exc()
        continue

# =====================================================
# COMPARATIVE ANALYSIS (if both datasets evaluated)
# =====================================================

if len(all_evaluation_results) == 2 and 'v7' in all_evaluation_results and 'v8' in all_evaluation_results:
    print("\n" + "=" * 70)
    print("COMPARATIVE ANALYSIS: AF (v7) vs KFS (v8)")
    print("=" * 70)

    v7_results = all_evaluation_results['v7']['evaluation']
    v8_results = all_evaluation_results['v8']['evaluation']

    print("\nOverall Performance Comparison:")
    print(f"{'Metric':<20} {'AF (v7)':<15} {'KFS (v8)':<15} {'Difference':<15}")
    print("-" * 65)

    metrics_to_compare = ['accuracy', 'macro_precision', 'macro_recall', 'macro_f1', 'macro_auc']

    for metric in metrics_to_compare:
        v7_val = v7_results['overall_performance'][metric]
        v8_val = v8_results['overall_performance'][metric]
        diff = v8_val - v7_val

        print(f"{metric:<20} {v7_val:<15.4f} {v8_val:<15.4f} {diff:+.4f}")

    print(f"\nEvaluation Modes:")
    print(f"  AF (v7): {v7_results['evaluation_metadata']['evaluation_mode']}")
    print(f"  KFS (v8): {v8_results['evaluation_metadata']['evaluation_mode']}")

    if 'kfs_late_fusion_info' in v8_results:
        print(f"\nKFS Late Fusion Strategy:")
        print(f"  Frames used: {v8_results['kfs_late_fusion_info']['total_frames']}")
        print(f"  Video predictions: {v8_results['kfs_late_fusion_info']['total_videos']}")
        print(f"  Aggregation: {v8_results['kfs_late_fusion_info']['aggregation_method']}")

# Memory cleanup
if torch.cuda.is_available():
    torch.cuda.synchronize()
    torch.cuda.empty_cache()

print("\n" + "=" * 70)
print("CASME II ViT EVALUATION COMPLETED")
print("=" * 70)
print(f"Evaluated datasets: {EVALUATE_DATASETS}")
print("Next: Cell 4 - Generate confusion matrices and visualization")

CASME II ViT Evaluation Framework with Dual Dataset Support
Datasets to evaluate: ['v7', 'v8']

EVALUATING DATASET: V7

Test Dataset Configuration:
  Version: v7
  Variant: AF
  Description: Apex Frame with Face-Aware Preprocessing
  Frame strategy: apex_frame
  Evaluation mode: frame_level
  Dataset path: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/datasets/processed_casme2/preprocessed_v7

Creating CASME II test dataset from AF...
Loading CASME II test dataset for evaluation...
Found 28 image files in directory
Loaded 28 CASME II test samples for evaluation
Test set class distribution:
  others: 10 samples (35.7%)
  disgust: 7 samples (25.0%)
  happiness: 4 samples (14.3%)
  repression: 3 samples (10.7%)
  surprise: 3 samples (10.7%)
  sadness: 1 samples (3.6%)
Unique video IDs: 28
Missing classes in test set: ['fear']
Preloading 28 test images to RAM with 32 workers...


Loading test to RAM: 100%|██████████| 28/28 [00:01<00:00, 23.63it/s]


TEST RAM caching completed: 28/28 images, ~0.02GB
Loading trained model from: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/models/06_01_vit_casme2_kfs_prep/casme2_vit_kfs_best_f1.pth


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ViT feature dimension: 768
ViT CASME II Simplified: 768 -> 256 -> 7
Dropout rate: 0.3 (increased for regularization)
Model loaded successfully from epoch 0
Training best val F1: 0.1691

Running frame-level evaluation for AF...
Running frame-level inference...


...

TEST RAM caching completed: 84/84 images, ~0.05GB
Loading trained model from: /content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project/models/06_01_vit_casme2_kfs_prep/casme2_vit_kfs_best_f1.pth


Some weights of the model checkpoint at google/vit-base-patch16-224-in21k were not used when initializing ViTModel: ['pooler.dense.bias', 'pooler.dense.weight']
- This IS expected if you are initializing ViTModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ViTModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


ViT feature dimension: 768
ViT CASME II Simplified: 768 -> 256 -> 7
Dropout rate: 0.3 (increased for regularization)
Model loaded successfully from epoch 0
Training best val F1: 0.1691

Running video-level evaluation with late fusion for KFS...
Running frame-level inference for late fusion...


Processing frames: 100%|██████████| 11/11 [00:00<00:00, 16.31it/s]

Applying late fusion across 84 videos...
Late fusion completed: 84 video predictions from 84 frames
Inference time: 0.68s
Evaluation results saved:
  Main results: casme2_vit_evaluation_results_v8.json
  Wrong predictions: casme2_vit_wrong_predictions_v8.json

EVALUATION RESULTS - KFS (v8)

Overall Performance:
  Accuracy:  0.3690
  Precision: 0.1062
  Recall:    0.1701
  F1 Score:  0.1296
  AUC:       0.5390

Late Fusion Info:
  Total frames processed: 84
  Video-level predictions: 84
  Aggregation method: probability_averaging

Per-Class Performance:
  others [Present]: F1=0.5634, Support=30
  disgust [Present]: F1=0.3438, Support=21
  happiness [Present]: F1=0.0000, Support=12
  repression [Present]: F1=0.0000, Support=9
  surprise [Present]: F1=0.0000, Support=9
  sadness [Present]: F1=0.0000, Support=3
  fear [Missing]: F1=0.0000, Support=0

Wrong Predictions Analysis:
  Total errors: 53 / 84
  Error rate: 63.10%

Inference Performance:
  Total time: 0.68s
  Speed: 8.1 ms/sample



In [4]:
# @title Cell 4: CASME II ViT Confusion Matrix Generation

# File: 06_01_ViT_CASME2_KFS_Cell4.py
# Location: experiments/06_01_ViT_CASME2-KFS-PREP.ipynb
# Purpose: Generate professional confusion matrix visualization for AF and KFS evaluations

import json
import os
import glob
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
from datetime import datetime

print("CASME II ViT Confusion Matrix Generation")
print("=" * 60)

PROJECT_ROOT = "/content/drive/MyDrive/RESEARCH-WORKSPACE/ACTIVE-PROJECTS/Thesis_MER_Project"
RESULTS_ROOT = f"{PROJECT_ROOT}/results/06_01_vit_casme2_kfs_prep"

def find_evaluation_json_files(results_path):
    """Find evaluation JSON files with version detection"""
    json_files = {}
    eval_dir = f"{results_path}/evaluation_results"

    if os.path.exists(eval_dir):
        for version in ['v7', 'v8']:
            eval_pattern = f"{eval_dir}/casme2_vit_evaluation_results_{version}.json"
            eval_files = glob.glob(eval_pattern)

            if eval_files:
                json_files[f'main_{version}'] = eval_files[0]
                print(f"Found {version.upper()} evaluation file: {os.path.basename(eval_files[0])}")

            wrong_pattern = f"{eval_dir}/casme2_vit_wrong_predictions_{version}.json"
            wrong_files = glob.glob(wrong_pattern)

            if wrong_files:
                json_files[f'wrong_{version}'] = wrong_files[0]
                print(f"Found {version.upper()} wrong predictions: {os.path.basename(wrong_files[0])}")

        if not json_files:
            print(f"WARNING: No evaluation results found in {eval_dir}")
            print("Make sure Cell 3 (evaluation) has been executed first!")
    else:
        print(f"ERROR: Evaluation directory not found: {eval_dir}")

    return json_files

def load_evaluation_results(json_path):
    """Load and parse evaluation results JSON"""
    try:
        with open(json_path, 'r') as f:
            data = json.load(f)
        print(f"Successfully loaded: {os.path.basename(json_path)}")
        return data
    except Exception as e:
        print(f"ERROR loading {json_path}: {str(e)}")
        return None

def calculate_weighted_f1(per_class_performance):
    """Calculate weighted F1 score"""
    total_support = sum([class_data['support'] for class_data in per_class_performance.values()
                        if class_data['support'] > 0])

    if total_support == 0:
        return 0.0

    weighted_f1 = 0.0

    for class_name, class_data in per_class_performance.items():
        if class_data['support'] > 0:
            weight = class_data['support'] / total_support
            weighted_f1 += class_data['f1_score'] * weight

    return weighted_f1

def calculate_balanced_accuracy(confusion_matrix):
    """Calculate balanced accuracy handling classes with zero support"""
    cm = np.array(confusion_matrix)
    n_classes = cm.shape[0]

    per_class_balanced_acc = []
    classes_with_samples = []

    for i in range(n_classes):
        if cm[i, :].sum() > 0:
            classes_with_samples.append(i)

    for i in classes_with_samples:
        tp = cm[i, i]
        fn = cm[i, :].sum() - tp
        fp = cm[:, i].sum() - tp
        tn = cm.sum() - tp - fn - fp

        sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        specificity = tn / (tn + fp) if (tn + fp) > 0 else 0.0

        class_balanced_acc = (sensitivity + specificity) / 2
        per_class_balanced_acc.append(class_balanced_acc)

    balanced_acc = np.mean(per_class_balanced_acc) if per_class_balanced_acc else 0.0

    return balanced_acc

def determine_text_color(color_value, threshold=0.5):
    """Determine optimal text color based on background intensity"""
    return 'white' if color_value > threshold else 'black'

def create_confusion_matrix_plot(data, output_path, test_version):
    """Create professional confusion matrix visualization for CASME II micro-expression recognition"""

    meta = data['evaluation_metadata']
    class_names = meta['class_names']
    cm = np.array(data['confusion_matrix'], dtype=int)
    overall = data['overall_performance']
    per_class = data['per_class_performance']

    test_config = data.get('test_configuration', {})
    variant = test_config.get('variant', test_version.upper())
    description = test_config.get('description', f'{test_version} preprocessing')
    eval_mode = meta.get('evaluation_mode', 'frame_level')

    print(f"Processing confusion matrix for {variant} ({test_version})")
    print(f"Dataset: {description}")
    print(f"Evaluation mode: {eval_mode}")
    print(f"Confusion matrix shape: {cm.shape}")

    macro_f1 = overall.get('macro_f1', 0.0)
    accuracy = overall.get('accuracy', 0.0)
    weighted_f1 = calculate_weighted_f1(per_class)
    balanced_acc = calculate_balanced_accuracy(cm)

    print(f"Calculated metrics - Macro F1: {macro_f1:.4f}, Weighted F1: {weighted_f1:.4f}, "
          f"Balanced Acc: {balanced_acc:.4f}, Accuracy: {accuracy:.4f}")

    row_sums = cm.sum(axis=1, keepdims=True)
    with np.errstate(divide='ignore', invalid='ignore'):
        cm_pct = np.divide(cm, row_sums, where=(row_sums!=0))
        cm_pct = np.nan_to_num(cm_pct)

    fig, ax = plt.subplots(figsize=(12, 10))

    cmap = 'Blues'
    im = ax.imshow(cm_pct, interpolation='nearest', cmap=cmap, vmin=0.0, vmax=0.8)

    cbar = plt.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
    cbar.set_label('True Class Percentage', rotation=270, labelpad=15, fontsize=11)

    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            count = cm[i, j]

            if row_sums[i, 0] > 0:
                percentage = cm_pct[i, j] * 100
                text = f"{count}\n{percentage:.1f}%"
            else:
                text = f"{count}\n(N/A)"

            cell_value = cm_pct[i, j]
            text_color = determine_text_color(cell_value, threshold=0.4)

            ax.text(j, i, text, ha="center", va="center",
                   color=text_color, fontsize=9, fontweight='bold')

    ax.set_xticks(np.arange(len(class_names)))
    ax.set_yticks(np.arange(len(class_names)))
    ax.set_xticklabels(class_names, rotation=45, ha='right', fontsize=10)
    ax.set_yticklabels(class_names, fontsize=10)
    ax.set_xlabel("Predicted Label", fontsize=12, fontweight='bold')
    ax.set_ylabel("True Label", fontsize=12, fontweight='bold')

    preprocessing_note = f"Preprocessing: {description}\n"
    preprocessing_note += f"Dataset: {test_version}\n"
    preprocessing_note += f"Evaluation: {eval_mode.replace('_', ' ').title()}"

    if 'kfs_late_fusion_info' in data:
        fusion_info = data['kfs_late_fusion_info']
        preprocessing_note += f"\nFrames: {fusion_info['total_frames']}, Videos: {fusion_info['total_videos']}"

    missing_classes = meta.get('missing_classes', [])
    if missing_classes:
        preprocessing_note += f"\nMissing: {', '.join(missing_classes)}"

    ax.text(0.02, 0.98, preprocessing_note, transform=ax.transAxes, fontsize=8,
            verticalalignment='top', bbox=dict(boxstyle='round', facecolor='lightyellow', alpha=0.8))

    title = f"CASME II {variant} Micro-Expression Recognition - Face-Aware Preprocessing\n"
    title += f"Acc: {accuracy:.4f}  |  Macro F1: {macro_f1:.4f}  |  Weighted F1: {weighted_f1:.4f}  |  Balanced Acc: {balanced_acc:.4f}"
    ax.set_title(title, fontsize=12, pad=25, fontweight='bold')

    plt.tight_layout()
    plt.savefig(output_path, dpi=300, bbox_inches='tight', facecolor='white')
    plt.close(fig)

    print(f"Confusion matrix saved to: {os.path.basename(output_path)}")

    return {
        'accuracy': accuracy,
        'macro_f1': macro_f1,
        'weighted_f1': weighted_f1,
        'balanced_accuracy': balanced_acc,
        'missing_classes': missing_classes
    }

def generate_performance_summary(evaluation_data, wrong_predictions_data=None):
    """Generate comprehensive performance summary"""

    print("\n" + "=" * 60)
    print("CASME II MICRO-EXPRESSION RECOGNITION PERFORMANCE SUMMARY")
    print("=" * 60)

    overall = evaluation_data['overall_performance']
    meta = evaluation_data['evaluation_metadata']
    test_config = evaluation_data.get('test_configuration', {})

    variant = test_config.get('variant', 'N/A')

    print(f"Dataset: {meta['dataset']}")
    print(f"Variant: {variant}")
    print(f"Dataset version: {test_config.get('version', 'N/A')}")
    print(f"Preprocessing: {test_config.get('description', 'N/A')}")
    print(f"Test samples: {meta['test_samples']}")
    print(f"Model: {meta['model_type']}")
    print(f"Evaluation date: {meta['evaluation_timestamp']}")

    if 'kfs_late_fusion_info' in evaluation_data:
        fusion_info = evaluation_data['kfs_late_fusion_info']
        print(f"\nLate Fusion Information:")
        print(f"  Total frames: {fusion_info['total_frames']}")
        print(f"  Video predictions: {fusion_info['total_videos']}")
        print(f"  Aggregation: {fusion_info['aggregation_method']}")

    print(f"\nOverall Performance:")
    print(f"  Accuracy:         {overall['accuracy']:.4f}")
    print(f"  Macro Precision:  {overall['macro_precision']:.4f}")
    print(f"  Macro Recall:     {overall['macro_recall']:.4f}")
    print(f"  Macro F1:         {overall['macro_f1']:.4f}")
    print(f"  Macro AUC:        {overall['macro_auc']:.4f}")

    print(f"\nPer-Class Performance:")
    per_class = evaluation_data['per_class_performance']

    print(f"{'Class':<12} {'F1':<8} {'Precision':<10} {'Recall':<8} {'AUC':<8} {'Support':<8} {'In Test'}")
    print("-" * 65)

    for class_name, metrics in per_class.items():
        in_test = "Yes" if metrics['in_test_set'] else "No"
        print(f"{class_name:<12} {metrics['f1_score']:<8.4f} {metrics['precision']:<10.4f} "
              f"{metrics['recall']:<8.4f} {metrics['auc']:<8.4f} {metrics['support']:<8} {in_test}")

    if 'training_information' in evaluation_data:
        training = evaluation_data['training_information']
        print(f"\nTraining vs Test Comparison:")
        print(f"  Training Val F1:  {training['best_val_f1']:.4f}")
        print(f"  Test F1:          {overall['macro_f1']:.4f}")
        print(f"  Performance Gap:  {training['best_val_f1'] - overall['macro_f1']:+.4f}")
        print(f"  Best Epoch:       {training['best_epoch']}")

    missing_classes = meta.get('missing_classes', [])
    available_classes = meta.get('available_classes', [])

    print(f"\nClass Availability Analysis:")
    print(f"  Available classes: {len(available_classes)}/7")
    print(f"  Missing classes: {missing_classes if missing_classes else 'None'}")

    if wrong_predictions_data:
        wrong_meta = wrong_predictions_data['analysis_metadata']
        print(f"\nError Analysis:")
        print(f"  Total errors: {wrong_meta['total_wrong_predictions']}/{wrong_meta['total_samples']}")
        print(f"  Error rate: {wrong_meta['overall_error_rate']:.2f}%")

        patterns = wrong_predictions_data.get('confusion_patterns', {})
        if patterns:
            print(f"\nTop Confusion Patterns:")
            sorted_patterns = sorted(patterns.items(), key=lambda x: x[1], reverse=True)[:3]
            for pattern, count in sorted_patterns:
                print(f"  {pattern}: {count} cases")

    print(f"\nInference Performance:")
    inference = evaluation_data['inference_performance']
    print(f"  Total time: {inference['total_time_seconds']:.2f}s")
    print(f"  Speed: {inference['average_time_ms_per_sample']:.1f} ms/sample")

json_files = find_evaluation_json_files(RESULTS_ROOT)

if not json_files:
    print(f"ERROR: No evaluation JSON files found in {RESULTS_ROOT}")
    print("Make sure Cell 3 (evaluation) has been executed first!")
else:
    print(f"\nFound {len([k for k in json_files.keys() if k.startswith('main_')])} evaluation result(s)")

output_dir = f"{RESULTS_ROOT}/confusion_matrix_analysis"
Path(output_dir).mkdir(parents=True, exist_ok=True)

results_summary = {}
generated_files = []

for version in ['v7', 'v8']:
    main_key = f'main_{version}'
    wrong_key = f'wrong_{version}'

    if main_key in json_files:
        print(f"\n{'='*60}")
        print(f"Processing {version.upper()} Evaluation Results")
        print(f"{'='*60}")

        eval_data = load_evaluation_results(json_files[main_key])

        wrong_data = None
        if wrong_key in json_files:
            wrong_data = load_evaluation_results(json_files[wrong_key])

        if eval_data is not None:
            try:
                cm_output_path = os.path.join(output_dir, f"confusion_matrix_CASME2_ViT_{version}.png")
                metrics = create_confusion_matrix_plot(eval_data, cm_output_path, version)
                generated_files.append(cm_output_path)

                results_summary[version] = metrics

                print(f"\nSUCCESS: {version.upper()} confusion matrix generated successfully")
                print(f"Output file: {os.path.basename(cm_output_path)}")

                print(f"\nPerformance Metrics Summary:")
                print(f"  Accuracy:        {metrics['accuracy']:.4f}")
                print(f"  Macro F1:        {metrics['macro_f1']:.4f}")
                print(f"  Weighted F1:     {metrics['weighted_f1']:.4f}")
                print(f"  Balanced Acc:    {metrics['balanced_accuracy']:.4f}")

                if metrics['missing_classes']:
                    print(f"  Missing classes: {metrics['missing_classes']}")

            except Exception as e:
                print(f"ERROR: Failed to generate {version.upper()} confusion matrix: {str(e)}")
                import traceback
                traceback.print_exc()

            generate_performance_summary(eval_data, wrong_data)
        else:
            print(f"ERROR: Could not load {version.upper()} evaluation data")

if generated_files:
    print(f"\n" + "=" * 60)
    print("CASME II CONFUSION MATRIX GENERATION COMPLETED")
    print("=" * 60)

    print(f"\nGenerated visualization files:")
    for file_path in generated_files:
        filename = os.path.basename(file_path)
        print(f"  {filename}")

    for version in ['v7', 'v8']:
        if version in results_summary:
            variant = 'AF' if version == 'v7' else 'KFS'
            print(f"\n{variant} ({version.upper()}) Performance Summary:")
            metrics = results_summary[version]
            print(f"  Accuracy:       {metrics['accuracy']:.4f}")
            print(f"  Macro F1:       {metrics['macro_f1']:.4f}")
            print(f"  Weighted F1:    {metrics['weighted_f1']:.4f}")
            print(f"  Balanced Acc:   {metrics['balanced_accuracy']:.4f}")

    print(f"\nFiles saved in: {output_dir}")
    print(f"Analysis completed at: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

else:
    print(f"\nERROR: No visualizations were generated")
    print("Please check:")
    print("1. Cell 3 evaluation results exist")
    print("2. JSON file structure is correct")
    print("3. No file permission issues")

print("\nCell 4 completed - CASME II confusion matrix analysis generated")

CASME II ViT Confusion Matrix Generation
Found V7 evaluation file: casme2_vit_evaluation_results_v7.json
Found V7 wrong predictions: casme2_vit_wrong_predictions_v7.json
Found V8 evaluation file: casme2_vit_evaluation_results_v8.json
Found V8 wrong predictions: casme2_vit_wrong_predictions_v8.json

Found 2 evaluation result(s)

Processing V7 Evaluation Results
Successfully loaded: casme2_vit_evaluation_results_v7.json
Successfully loaded: casme2_vit_wrong_predictions_v7.json
Processing confusion matrix for AF (v7)
Dataset: Apex Frame with Face-Aware Preprocessing
Evaluation mode: frame_level
Confusion matrix shape: (7, 7)
Calculated metrics - Macro F1: 0.1241, Weighted F1: 0.2798, Balanced Acc: 0.5180, Accuracy: 0.3571
Confusion matrix saved to: confusion_matrix_CASME2_ViT_v7.png

SUCCESS: V7 confusion matrix generated successfully
Output file: confusion_matrix_CASME2_ViT_v7.png

Performance Metrics Summary:
  Accuracy:        0.3571
  Macro F1:        0.1241
  Weighted F1:     0.2798
